In [137]:
# Descarga tu repositorio desde GitHub
!git clone https://github.com/Wmarbar/CProgram_incidencias.git

# Cambia de directorio para acceder a los archivos
%cd CProgram_incidencias

# Verifica que los archivos estén
!ls


Cloning into 'CProgram_incidencias'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 48 (delta 24), reused 9 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (48/48), 1.64 MiB | 17.66 MiB/s, done.
Resolving deltas: 100% (24/24), done.
/content/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias
 constructor4_proyecto_II.ipynb   README.md
 input				 'Trabajo-CalidadDeDatos Ver4-2.pdf'
 logo_u.png


# __PROGRAMACIÓN PARA ANALÍTICA DE DATOS__

<img src="https://github.com/WMARBAR/CProgram_incidencias/blob/main/logo_u.png?raw=1" alt="Logo Universidad Central" width="300" />

> ## __PROYECTO: Estimación Área bajo la curva__

> ## Profesor: Jorge Victorino [jvictorinog@ucentral.edu.co](jvictorinog@ucentral.edu.co)<br>
> ## Estudiante: Sergio Andrés Sánchez Cárdenas [correo@ucentral.edu.co](mailto:correo@ucentral.edu.co)<br>
> ## Estudiante: Wilson Felipe Mártinez Barrantes [wmartinezb1@ucentral.edu.co](jvictorinog@ucentral.edu.co)<br>
> ## Grupo: 5 <br>
> ### Facultad de Ingeniería y Ciencias Básicas <br>
> ### Universidad Central <br>
> <br>

---


# FUNCTIONS

In [138]:
import pandas as pd
import numpy as np

def load_excel(path):

 df_incidentes = pd.read_excel(path)
 return df_incidentes


def delta_dates(df, col1, col2):
    try:
        # Verificar si las columnas existen
        if col1 not in df.columns or col2 not in df.columns:
            print(f"Las columnas '{col1}' o '{col2}' no existen en el DataFrame.")
            return df

        # Convertir columnas a datetime con coerción
        df[col1] = pd.to_datetime(df[col1], errors='coerce')
        df[col2] = pd.to_datetime(df[col2], errors='coerce')

        # Avisar si hay muchas fechas inválidas (pero continuar)
        if df[col1].isna().any():
            print(f"Algunos valores en '{col1}' no son fechas válidas y se marcarán como NaT.")
        if df[col2].isna().any():
            print(f"Algunos valores en '{col2}' no son fechas válidas y se marcarán como NaT.")

        # Calcular diferencia en días (donde ambas fechas sean válidas)
        nueva_col = f"{col1}_{col2}_days"
        df[nueva_col] = (df[col2] - df[col1]).dt.days

        return df

    except Exception as e:
        print(f"Error inesperado: {e}")
        return df

def key_generator(df, columnas):
    try:
        # Validar columnas
        for col in columnas:
            if col not in df.columns:
                print(f"La columna '{col}' no existe en el DataFrame.")
                return df

        # Nombre de la nueva columna
        nombre_key = '_'.join(columnas) + '_KEY'

        # Función de limpieza para cada valor
        def limpiar_valor(v):
            if isinstance(v, float) and v.is_integer():
                return str(int(v))
            return str(v)

        # Aplicar limpieza y concatenar
        df[nombre_key] = df[columnas].apply(lambda row: '_'.join(limpiar_valor(v) for v in row), axis=1)

        return df

    except Exception as e:
        print(f"Error inesperado: {e}")
        return df


def drop_nan_by_column(df, col):
    if col not in df.columns:
        print(f"La columna '{col}' no existe en el DataFrame.")
        return df

    df_limpio = df.dropna(subset=[col])
    return df_limpio


def deduplicar_incidentes(df, key_col='Incidente_Estructura_KEY', dias_col='Creacion_Inicio_days'):
    if key_col not in df.columns or dias_col not in df.columns:
        print(f"❌ Las columnas '{key_col}' o '{dias_col}' no existen.")
        return df, pd.DataFrame()

    indices_a_conservar = []

    # Función para aplicar por grupo
    def resolver_grupo(grupo):
        sin_cero = grupo[grupo[dias_col] != 0]
        if not sin_cero.empty:
            idx = sin_cero.index[0]
        else:
            idx = grupo.index[0]
        indices_a_conservar.append(idx)
        return

    # Agrupar y marcar qué índices se deben conservar
    df.groupby(key_col, group_keys=False).apply(resolver_grupo)

    # Separar los que se quedan y los que se van
    df_filtrado = df.loc[indices_a_conservar].reset_index(drop=True)
    df_duplits = df.drop(index=indices_a_conservar).reset_index(drop=True)

    print(f"✅ Registros conservados: {len(df_filtrado)} | Registros duplicados removidos: {len(df_duplits)}")

    return df_filtrado, df_duplits


def unique_validator(df, col):
    if col not in df.columns:
        print(f"La columna '{col}' no existe en el DataFrame.")
        return

    duplicados = df.duplicated(subset=[col]).sum()

    if duplicados > 0:
        print(f"Hay {duplicados} valores duplicados en la columna '{col}'.")
    else:
        print(f"Todos los valores en la columna '{col}' son únicos.")



def nan_detector_column(df, col):
    if col not in df.columns:
        print(f"La columna '{col}' no existe en el DataFrame.")
        return df

    df[f'{col}_NAN_DETECTOR'] = df[col].apply(lambda x: 1 if pd.isna(x) or (isinstance(x, str) and x.strip() == '') else 0)

    return df

def deduplicar_estructuras_por_vereda(df, key_col='Estructura', prioridad_col='Vereda'):
    # Validar que las columnas existan
    if key_col not in df.columns or prioridad_col not in df.columns:
        print(f"Las columnas '{key_col}' o '{prioridad_col}' no existen.")
        return df

    # Función para aplicar por grupo
    def resolver_grupo(grupo):
        con_vereda = grupo[grupo[prioridad_col].apply(lambda x: isinstance(x, str) and x.strip() != '') | grupo[prioridad_col].notna()]
        if not con_vereda.empty:
            return con_vereda.iloc[[0]]  # Uno cualquiera con vereda
        else:
            return grupo.iloc[[0]]  # Si no hay vereda, igual uno cualquiera

    # Aplicar la lógica por grupo
    df_filtrado = df.groupby(key_col, group_keys=False).apply(resolver_grupo).reset_index(drop=True)

    return df_filtrado


def limpiar_nombres_columnas(df):
    df.columns = df.columns.str.strip().str.replace(r'\s+', '_', regex=True)
    return df

def corregir_basura_columna(df, col):
    if col not in df.columns:
        print(f"La columna '{col}' no existe en el DataFrame.")
        return df

    def _fix_encoding(x):
        if isinstance(x, str):
            try:
                return x.encode('latin1').decode('utf-8')
            except Exception:
                return x
        return x

    df[col] = df[col].apply(_fix_encoding)
    return df

def merge_excluir_llave(df1, df2, llave_df1, llave_df2):
    # Verificar existencia de las columnas
    if llave_df1 not in df1.columns or llave_df2 not in df2.columns:
        print(f"Las llaves '{llave_df1}' o '{llave_df2}' no existen.")
        return df1

    # Hacer el merge completo primero
    df_merged = df1.merge(df2, left_on=llave_df1, right_on=llave_df2, how='left')

    # Eliminar la llave de df2 si es distinta a la de df1
    if llave_df1 != llave_df2:
        df_merged = df_merged.drop(columns=[llave_df2])

    return df_merged

def guardar_excel(df, nombre_archivo='salida.xlsx', sheet_name='Hoja1'):
    try:
        df.to_excel(nombre_archivo, index=False, sheet_name=sheet_name)
        print(f"Archivo guardado exitosamente como '{nombre_archivo}'.")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")


def marcar_estado_interes(df, col='Estado'):
    palabras_clave = ['GPRE', 'PRER', 'TAMB', 'GEAM', 'AMPO', 'GERE']
    # Usamos regex para buscar cualquiera de las palabras
    patron = '|'.join(palabras_clave)

    df['estado_interes'] = df[col].astype(str).str.contains(patron).astype(int)

    return df

def ones_selector(df, col):
    if col not in df.columns:
        print(f"La columna '{col}' no existe en el DataFrame.")
        return df

    df_filtrado = df[df[col] == 1].copy()
    return df_filtrado

def spliter(df, col):
    if col not in df.columns:
        print(f"La columna '{col}' no existe en el DataFrame.")
        return df

    # Separar por espacio
    nuevas_cols = df[col].astype(str).str.split(expand=True)

    # Renombrar columnas nuevas con sufijos: col_1, col_2, etc.
    nuevas_cols.columns = [f"{col}_{i+1}" for i in range(nuevas_cols.shape[1])]

    # Unir al DataFrame original
    df_expandido = pd.concat([df, nuevas_cols], axis=1)

    return df_expandido

def contar_columnas_con_dato(df, columnas):
    # Validar que las columnas existan
    for col in columnas:
        if col not in df.columns:
            print(f"La columna '{col}' no existe en el DataFrame.")
            return df

    # Contar columnas con dato no nulo y no vacío
    df['Estados'] = df[columnas].apply(
        lambda fila: sum(1 for x in fila if pd.notna(x) and (not isinstance(x, str) or x.strip() != '')),
        axis=1
    )

    return df

# CARGA DE ARCHIVOS

In [139]:

df_incidentes = load_excel("./input/incidentes1.xlsx")
df_estructura = load_excel("./input/estructura1.xlsx")

# DEV ZONE INCIDENCIAS

## ARREGLO DE LA DATA INCIDENCIAS

In [140]:
df_incidentes.columns
df_incidentes=marcar_estado_interes(df_incidentes,'Estado')
df_incidentes=ones_selector(df_incidentes,'estado_interes')

In [141]:

df_incidentes=delta_dates(df_incidentes,'Creacion','Inicio')
df_incidentes=delta_dates(df_incidentes,'Creacion','Cierre')
df_incidentes=delta_dates(df_incidentes,'Inicio','Cierre')
df_incidentes=key_generator(df_incidentes,['Incidente','Estructura'])
df_incidentes=drop_nan_by_column(df_incidentes,'Estructura')
df_incidentes, df_incidentes_dups =deduplicar_incidentes(df_incidentes)
df_incidentes=nan_detector_column(df_incidentes,'Cierre')
df_incidentes=limpiar_nombres_columnas(df_incidentes)
df_incidentes['incidentes_counter']=1
df_incidentes.isna().sum()

Algunos valores en 'Cierre' no son fechas válidas y se marcarán como NaT.
Algunos valores en 'Cierre' no son fechas válidas y se marcarán como NaT.
✅ Registros conservados: 3118 | Registros duplicados removidos: 1398


<ipython-input-138-3271a2c709b1>:91: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby(key_col, group_keys=False).apply(resolver_grupo)


,0
Incidente,0
Estructura,0
Tipo_incidente,0
Estado,0
Prioridad,17
Creacion,0
Inicio,0
Cierre,743
estado_interes,0
Creacion_Inicio_days,0


# DEV ZONE Estructura

## ARREGLO DE LA DATA ESTRUCTURAS

In [142]:

df_estructura= limpiar_nombres_columnas(df_estructura)
df_estructura= drop_nan_by_column(df_estructura,'Estructura')
df_estructura= deduplicar_estructuras_por_vereda(df_estructura)
df_estructura = corregir_basura_columna(df_estructura, 'Depto')
df_estructura['Depto'] = df_estructura['Depto'].replace('NARIÃ‘O', 'NARIÑO')
df_estructura = corregir_basura_columna(df_estructura, 'Municipio')
df_estructura['Municipio'] = df_estructura['Municipio'].replace('IBAGUÃ‰', 'IBAGUÉ')
df_estructura['Municipio'] = df_estructura['Municipio'].replace('JERUSALÃ‰N', 'JERUSALÉN')
df_estructura['Municipio'] = df_estructura['Municipio'].replace('GUACHENÃ‰', 'GUACHENÉ')
df_estructura['Latitud'] = df_estructura['Latitud'].replace(',', '.')
df_estructura['Longitud'] = df_estructura['Longitud'].replace(',', '.')

<ipython-input-138-3271a2c709b1>:140: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtrado = df.groupby(key_col, group_keys=False).apply(resolver_grupo).reset_index(drop=True)


In [143]:
df_incidentes.columns

Index(['Incidente', 'Estructura', 'Tipo_incidente', 'Estado', 'Prioridad',
       'Creacion', 'Inicio', 'Cierre', 'estado_interes',
       'Creacion_Inicio_days', 'Creacion_Cierre_days', 'Inicio_Cierre_days',
       'Incidente_Estructura_KEY', 'Cierre_NAN_DETECTOR',
       'incidentes_counter'],
      dtype='object')

# DEV ZONE Base de incidencias completa

In [144]:
df_merged= merge_excluir_llave(df_incidentes,df_estructura,'Estructura','Estructura')
df_merged=nan_detector_column(df_merged,'Serie')


In [150]:
df_merged=spliter(df_merged,'Estado')
df_merged=contar_columnas_con_dato(df_merged, ['Estado_1', 'Estado_2', 'Estado_3','Estado_4','Estado_5','Estado_6','Estado_7'])
df_merged['Estructura']=df_merged['Estructura'].astype(int)
df_merged['Creacion_Cierre_days']=df_merged['Creacion_Cierre_days'].astype(int)
df_merged['Inicio_Cierre_days']=df_merged['Inicio_Cierre_days'].astype(int)
df_merged.dtypes



,0
Incidente,int64
Estructura,int64
Tipo_incidente,object
Estado,object
Prioridad,object
Creacion,datetime64[ns]
Inicio,datetime64[ns]
Cierre,datetime64[ns]
estado_interes,int64
Creacion_Inicio_days,int64


In [149]:
df_merged.dtypes

,0
Incidente,int64
Estructura,float64
Tipo_incidente,object
Estado,object
Prioridad,object
Creacion,datetime64[ns]
Inicio,datetime64[ns]
Cierre,datetime64[ns]
estado_interes,int64
Creacion_Inicio_days,int64


In [146]:
guardar_excel(df_merged)

Archivo guardado exitosamente como 'salida.xlsx'.


In [147]:
from google.colab import files
files.download('salida.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>